# Sample MERRA-2 Along a CALIPSO Obrit
This is an example showing how to use GMAOpyobs utilities to sample MERRA-2 along a CALIPSO orbit.

This assumes that you have cloned GMAOpyobs Github repository, installed it in a directory called \\$AERODIR, and have added the following directory to your $PYTHONPATh environment variable:
\\$AERODIR/install/lib/Python

It also assumes you have a grads style control file for the MERRA-2 files. An example of such a file is in this directory called inst3_3d_aer_Nv

This MERRA-2 Collection containes 3-D aerosol mass mixing ratios


## CALIPSO Example

In [1]:
from pyobs.sampler import TRAJECTORY
from pyobs.calipso_l1p5 import CALIPSO_L1p5

In [11]:
# set up some file names
calipsoFile = 'CAL_LID_L15-Standard-V1-00.2008-11-11T04-34-35ZN.hdf'
m2data = 'tavg1_2d_aer_Nx'
outFile = 'm2_calipso_sampled.nc4'

In [3]:
# Read the CALIPSO file, get the lat, lon, times from the file
caldata = CALIPSO_L1p5(calipsoFile)

shape Latitude (924,)
shape Longitude (924,)
shape Surface_Elevation_Mean (924,)
shape Profile_UTC_Time (924,)
shape Pressure (924, 400)
shape Temperature (924, 400)
shape Total_Attenuated_Backscatter_532_Mean (924, 400)
shape Total_Attenuated_Backscatter_Uncertainty_532 (924, 400)
shape Perpendicular_Attenuated_Backscatter_532_Mean (924, 400)
shape Perpendicular_Attenuated_Backscatter_Uncertainty_532 (924, 400)
shape Molecular_Model_Attenuated_Backscatter_532 (924, 400)
shape L2_Feature_Type (924, 400, 4)
shape nymd (853776,)
shape nhms (853776,)


In [6]:
# create a trajectory oject
traj = TRAJECTORY(caldata.time,caldata.lon,caldata.lat,m2data)

In [13]:
# sample the MERRA-2 dataset along the trajectory, and return an xarray dataset
traj_ds = traj.sample()

In [14]:
# write data to a netcdf file
traj_ds.to_netcdf(outFile)

# Calculate Aerosol Optical Properties on the Sampled Dataset
Now that we have the 3-D aerosol mass mixing ratios, we would like to convert this to 3-D total and speciated aerosol optical properties.
You will need the aerosol optics tables, and a yaml file describing which files to use.
An example yaml file is in this directory called Aod_EOS.yaml

In [25]:
from pyobs.aop import G2GAOP

In [26]:
# set up some filenames
config = 'Aod_EOS.yaml'
aerFile = 'm2_calipso_sampled.nc4'

In [28]:
# read the aerosol data and optical tables
optics = G2GAOP(aerFile,config=config)